In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [2]:
#list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'stretch_padding', 'ext_padding', 
#                 'rdm_padding', 'aug_padding']
#list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
#                 'rnd_padding']
list_paddings = ["aug_padding"]

#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/3denses/bio_neurons/'
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 7 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [313,76]
drop_hid = 0.5
dict_size = len(dicti)
final_act = "softmax"

batch_size = 54
epochss = 200

callbacks_list = [None, None, None, None, False, False, False, False]
task = "task2/"
architecture = "only_denses"

### Loading stuff

In [3]:
###LOADING ENZYME INDICES BECAUSE THIS IS TASK 2

file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_enz.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_data_enz.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

# Defining Deep Learning model - Task 2

In [4]:
model = building_2dense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)

generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task2', True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 26000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 313)               8138313   
_________________________________________________________________
dropout_2 (Dropout)          (None, 313)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 76)                23864     
_________________________________________________________________
dropout_3 (Dropout)          (None, 76)                0         
__________

### Training model

In [ ]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        for idx,i in enumerate(generators):
            if model_type == "aug_padding":
                i_train, i_val, i_test = k_aug_indices[idx]
            else: 
                i_train, i_val, i_test = splitting_sets[idx]
            len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
            train_generator, val_generator = i
            model_training(model_type, folder, task, idx, callbacks_list, train_generator, 
                           val_generator, architecture, max_lenn, dict_size, batch_size, 
                           n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                           len_train, len_val,
                           n_filt = None, kernel_size=None, pool_size=None)

In [ ]:
# BORRAR CUANDO COMPRUEBE QUE LO OTRO VA BIEN

task = "task2/"
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else:
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        folder_cp = ''.join(string for string in [folder, task, model_type, '/', str(idx)]) 
        #print(folder_cp)
        if not os.path.exists(os.path.join(absPath, 'data/checkpoint/', folder_cp)):
            os.makedirs(''.join(string for string in [absPath, 'data/checkpoint/', folder_cp]))

        callbacks_list = calling_callbacks(folder_cp, folder, model_type, None, None, None, 
                                   None, False, False, False, False)
        folder_task1 =  ''.join(string for string in [folder, task])
        model = building_2dense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)
        #writing log file 
        log_file = ''.join(string for string in [absPath, 'data/checkpoint/', folder, task, 'log_file.txt' ]) 
        f = open(log_file, 'a+')
        print('Model type: %s \n' % model_type, file=f)
        print('Fold: %i \n' % idx, file=f)
 
        start = time.time()
        formatted_time = datetime.datetime.now()
        print('Starting time: %s \n' % formatted_time, file=f)
        history = model.fit_generator(generator=train_generator, 
                              validation_data=val_generator,
                             steps_per_epoch= int(len_train/batch_size),
                              validation_steps=int(len_val/batch_size),
                             epochs=epochss,
                             callbacks=callbacks_list,
                             verbose=1)
        end = time.time()
        formatted_endtime = datetime.datetime.now()
        print('Finishing time: %s \n' % formatted_endtime, file=f)
        count_time(start, end, folder, model_type)
        saving_results(history, model_type, folder, idx, True)
        f.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 26000)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 313)               8138313   
_________________________________________________________________
dropout_5 (Dropout)          (None, 313)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 76)                23864     
_________________________________________________________________
dropout_6 (Dropout)          (None, 76)                0         
__________

138/138 [==============================] - 2s 15ms/step - loss: 0.4871 - acc: 0.2525 - val_loss: 0.1367 - val_acc: 0.1737

Epoch 00030: val_acc did not improve from 0.20179
Epoch 31/200
138/138 [==============================] - 2s 14ms/step - loss: 0.4936 - acc: 0.2609 - val_loss: 0.1329 - val_acc: 0.1935

Epoch 00031: val_acc did not improve from 0.20179
Epoch 32/200
138/138 [==============================] - 2s 15ms/step - loss: 0.4889 - acc: 0.2564 - val_loss: 0.1444 - val_acc: 0.1731

Epoch 00032: val_acc did not improve from 0.20179
Epoch 33/200
138/138 [==============================] - 2s 15ms/step - loss: 0.4798 - acc: 0.2594 - val_loss: 0.1334 - val_acc: 0.2229

Epoch 00033: val_acc improved from 0.20179 to 0.22286, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task2/post_padding/0/weights-improvement-033-0.2229.hdf5
Epoch 34/200
138/138 [==============================] - 2s 15ms/step - loss: 0.4738 - acc: 0.2728 - val_loss: 0.

138/138 [==============================] - 2s 15ms/step - loss: 0.3401 - acc: 0.3392 - val_loss: 0.0827 - val_acc: 0.2088

Epoch 00069: val_acc did not improve from 0.28991
Epoch 70/200
138/138 [==============================] - 2s 15ms/step - loss: 0.3338 - acc: 0.3407 - val_loss: 0.0952 - val_acc: 0.2139

Epoch 00070: val_acc did not improve from 0.28991
Epoch 71/200
138/138 [==============================] - 2s 15ms/step - loss: 0.3157 - acc: 0.3457 - val_loss: 0.0809 - val_acc: 0.2114

Epoch 00071: val_acc did not improve from 0.28991
Epoch 72/200
138/138 [==============================] - 2s 14ms/step - loss: 0.3515 - acc: 0.3438 - val_loss: 0.0889 - val_acc: 0.2158

Epoch 00072: val_acc did not improve from 0.28991
Epoch 73/200
138/138 [==============================] - 2s 14ms/step - loss: 0.3147 - acc: 0.3468 - val_loss: 0.0740 - val_acc: 0.2433

Epoch 00073: val_acc did not improve from 0.28991
Epoch 74/200
138/138 [==============================] - 2s 14ms/step - loss: 0.3110

138/138 [==============================] - 2s 15ms/step - loss: 0.2567 - acc: 0.3775 - val_loss: 0.0564 - val_acc: 0.2407

Epoch 00113: val_acc did not improve from 0.28991
Epoch 114/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2509 - acc: 0.3862 - val_loss: 0.0662 - val_acc: 0.2324

Epoch 00114: val_acc did not improve from 0.28991
Epoch 115/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2483 - acc: 0.3874 - val_loss: 0.0532 - val_acc: 0.2510

Epoch 00115: val_acc did not improve from 0.28991
Epoch 116/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2497 - acc: 0.3873 - val_loss: 0.0529 - val_acc: 0.2152

Epoch 00116: val_acc did not improve from 0.28991
Epoch 117/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2416 - acc: 0.3862 - val_loss: 0.0686 - val_acc: 0.2299

Epoch 00117: val_acc did not improve from 0.28991
Epoch 118/200
138/138 [==============================] - 2s 15ms/step - loss: 0

138/138 [==============================] - 2s 15ms/step - loss: 0.2339 - acc: 0.4004 - val_loss: 0.0656 - val_acc: 0.1928

Epoch 00157: val_acc did not improve from 0.28991
Epoch 158/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2322 - acc: 0.3983 - val_loss: 0.0479 - val_acc: 0.2433

Epoch 00158: val_acc did not improve from 0.28991
Epoch 159/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2142 - acc: 0.4014 - val_loss: 0.0506 - val_acc: 0.1973

Epoch 00159: val_acc did not improve from 0.28991
Epoch 160/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2271 - acc: 0.3986 - val_loss: 0.0456 - val_acc: 0.2056

Epoch 00160: val_acc did not improve from 0.28991
Epoch 161/200
138/138 [==============================] - 2s 15ms/step - loss: 0.2260 - acc: 0.3937 - val_loss: 0.0465 - val_acc: 0.2101

Epoch 00161: val_acc did not improve from 0.28991
Epoch 162/200
138/138 [==============================] - 2s 15ms/step - loss: 0

...

138/138 [==============================] - 2s 16ms/step - loss: 0.3712 - acc: 0.3343 - val_loss: 0.1122 - val_acc: 0.1928

Epoch 00107: val_acc did not improve from 0.21903
Epoch 108/200
138/138 [==============================] - 2s 15ms/step - loss: 0.3774 - acc: 0.3313 - val_loss: 0.1112 - val_acc: 0.1852

Epoch 00108: val_acc did not improve from 0.21903
Epoch 109/200
138/138 [==============================] - 2s 15ms/step - loss: 0.3767 - acc: 0.3332 - val_loss: 0.1159 - val_acc: 0.2018

Epoch 00109: val_acc did not improve from 0.21903
Epoch 110/200
138/138 [==============================] - 2s 15ms/step - loss: 0.3624 - acc: 0.3392 - val_loss: 0.1102 - val_acc: 0.1960

Epoch 00110: val_acc did not improve from 0.21903
Epoch 111/200
138/138 [==============================] - 2s 15ms/step - loss: 0.3589 - acc: 0.3422 - val_loss: 0.1082 - val_acc: 0.1916

Epoch 00111: val_acc did not improve from 0.21903
Epoch 112/200
138/138 [==============================] - 2s 16ms/step - loss: 0

138/138 [==============================] - 2s 15ms/step - loss: 0.3482 - acc: 0.3430 - val_loss: 0.0999 - val_acc: 0.1928

Epoch 00151: val_acc did not improve from 0.21903
Epoch 152/200
138/138 [==============================] - 2s 15ms/step - loss: 0.3335 - acc: 0.3535 - val_loss: 0.0999 - val_acc: 0.1928

Epoch 00152: val_acc did not improve from 0.21903
Epoch 153/200
138/138 [==============================] - 2s 15ms/step - loss: 0.3483 - acc: 0.3441 - val_loss: 0.1003 - val_acc: 0.1935

Epoch 00153: val_acc did not improve from 0.21903
Epoch 154/200
138/138 [==============================] - 2s 16ms/step - loss: 0.3442 - acc: 0.3474 - val_loss: 0.1012 - val_acc: 0.1928

Epoch 00154: val_acc did not improve from 0.21903
Epoch 155/200
138/138 [==============================] - 2s 16ms/step - loss: 0.3360 - acc: 0.3523 - val_loss: 0.1012 - val_acc: 0.1954

Epoch 00155: val_acc did not improve from 0.21903
Epoch 156/200
138/138 [==============================] - 2s 16ms/step - loss: 0

138/138 [==============================] - 2s 15ms/step - loss: 0.3336 - acc: 0.3528 - val_loss: 0.1071 - val_acc: 0.1954

Epoch 00195: val_acc did not improve from 0.21903
Epoch 196/200
138/138 [==============================] - 2s 15ms/step - loss: 0.3220 - acc: 0.3551 - val_loss: 0.1098 - val_acc: 0.1935

Epoch 00196: val_acc did not improve from 0.21903
Epoch 197/200
138/138 [==============================] - 2s 16ms/step - loss: 0.3207 - acc: 0.3519 - val_loss: 0.1064 - val_acc: 0.1954

Epoch 00197: val_acc did not improve from 0.21903
Epoch 198/200
138/138 [==============================] - 2s 16ms/step - loss: 0.3140 - acc: 0.3572 - val_loss: 0.1047 - val_acc: 0.1973

Epoch 00198: val_acc did not improve from 0.21903
Epoch 199/200
138/138 [==============================] - 2s 16ms/step - loss: 0.3156 - acc: 0.3568 - val_loss: 0.1071 - val_acc: 0.1960

Epoch 00199: val_acc did not improve from 0.21903
Epoch 200/200
138/138 [==============================] - 2s 15ms/step - loss: 0

138/138 [==============================] - 2s 15ms/step - loss: 0.4014 - acc: 0.3164 - val_loss: 0.0776 - val_acc: 0.2056

Epoch 00024: val_acc did not improve from 0.21584
Epoch 25/200
138/138 [==============================] - 2s 16ms/step - loss: 0.4001 - acc: 0.3121 - val_loss: 0.0524 - val_acc: 0.1922

Epoch 00025: val_acc did not improve from 0.21584
Epoch 26/200
138/138 [==============================] - 2s 16ms/step - loss: 0.3737 - acc: 0.3241 - val_loss: 0.0616 - val_acc: 0.1897

Epoch 00026: val_acc did not improve from 0.21584
Epoch 27/200
138/138 [==============================] - 2s 16ms/step - loss: 0.3684 - acc: 0.3402 - val_loss: 0.0476 - val_acc: 0.1992

Epoch 00027: val_acc did not improve from 0.21584
Epoch 28/200
138/138 [==============================] - 2s 16ms/step - loss: 0.3704 - acc: 0.3413 - val_loss: 0.0482 - val_acc: 0.1935

Epoch 00028: val_acc did not improve from 0.21584
Epoch 29/200
138/138 [==============================] - 2s 16ms/step - loss: 0.3541

138/138 [==============================] - 2s 15ms/step - loss: 0.1817 - acc: 0.4675 - val_loss: 0.0179 - val_acc: 0.1992

Epoch 00109: val_acc did not improve from 0.27778
Epoch 110/200
138/138 [==============================] - 2s 15ms/step - loss: 0.1879 - acc: 0.4591 - val_loss: 0.0206 - val_acc: 0.2043

Epoch 00110: val_acc did not improve from 0.27778
Epoch 111/200
138/138 [==============================] - 2s 16ms/step - loss: 0.1747 - acc: 0.4842 - val_loss: 0.0222 - val_acc: 0.2043

Epoch 00111: val_acc did not improve from 0.27778
Epoch 112/200
138/138 [==============================] - 2s 16ms/step - loss: 0.1710 - acc: 0.4722 - val_loss: 0.0204 - val_acc: 0.2050

Epoch 00112: val_acc did not improve from 0.27778
Epoch 113/200
138/138 [==============================] - 2s 16ms/step - loss: 0.1779 - acc: 0.4800 - val_loss: 0.0190 - val_acc: 0.2037

Epoch 00113: val_acc did not improve from 0.27778
Epoch 114/200
138/138 [==============================] - 2s 16ms/step - loss: 0